In [15]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [16]:
actions = [
    'Hello',
    'Meet',
    'NiceTMY',
    'I',
    'Name',
    'You',
    'Today',
    'Weather',
    'Good',
    'Thankyou',
    'Sorry'
]

data = np.concatenate([
    np.load('dataset/seq_Hello_1716742569.npy'),
    np.load('dataset/seq_Meet_1716741759.npy'),
    np.load('dataset/seq_NiceTMY_1716741759.npy'),
    np.load('dataset/seq_I_1715840691.npy'),
    np.load('dataset/seq_Name_1715840691.npy'),
    np.load('dataset/seq_You_1716591701.npy'),
    np.load('dataset/seq_Today_1716741759.npy'),
    np.load('dataset/seq_Weather_1716741759.npy'),
    np.load('dataset/seq_Good_1716741759.npy'),
    np.load('dataset/seq_Thankyou_1716591701.npy'),
    np.load('dataset/seq_Sorry_1716591701.npy')
], axis=0)

data.shape

(8195, 30, 100)

In [17]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(8195, 30, 99)
(8195,)


In [18]:
import tensorflow as tf

num_classes = len(actions)

y_data = tf.one_hot(labels.astype(np.int32), depth=num_classes)
print(y_data.shape)

(8195, 11)


In [19]:
from sklearn.model_selection import train_test_split

# Convert data to NumPy arrays
x_data = x_data.astype(np.float32)
y_data = y_data.numpy()  # Convert tensor to NumPy array

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(7375, 30, 99) (7375, 11)
(820, 30, 99) (820, 11)


In [20]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

model = Sequential([
    Input(shape=x_train.shape[1:3]),
    LSTM(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        41,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 11)             │           363 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,427 (173.54 KB)

 Trainable params: 44,427 (173.54 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
# from tensorflow import load_model
import tensorflow as tf

# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)

# Save the model
model.save('models/model.h5')

model_path = 'models/model.h5'
model = tf.keras.models.load_model(model_path, compile=False)

y_pred = model.predict(x_val)

# Convert predictions and true labels to integer indices
y_val_int = np.argmax(y_val, axis=1).astype(np.int32)
y_pred_int = np.argmax(y_pred, axis=1).astype(np.int32)

# Ensure all classes are represented in the confusion matrix and classification report
labels_in_data = sorted(set(y_val_int).union(set(y_pred_int)))

# Compute confusion matrix
conf_matrix = confusion_matrix(y_val_int, y_pred_int, labels=labels_in_data)
print(conf_matrix)

# Classification report
class_report = classification_report(y_val_int, y_pred_int, target_names=[actions[i] for i in labels_in_data])
print(class_report)
# def multilabel_confusion_matrix(y_true, y_pred):
#     pass
# 
# 
# multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - acc: 0.2627 - loss: 30.1815 - val_acc: 0.3805 - val_loss: 8.3176
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - acc: 0.5317 - loss: 6.5419 - val_acc: 0.5927 - val_loss: 2.5595
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - acc: 0.6361 - loss: 2.1034 - val_acc: 0.6793 - val_loss: 1.5324
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - acc: 0.7116 - loss: 1.2781 - val_acc: 0.7061 - val_loss: 1.1725
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - acc: 0.3562 - loss: 24.9815 - val_acc: 0.4134 - val_loss: 5.2974
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - acc: 0.4723 - loss: 4.1427 - val_acc: 0.5671 - val_loss: 2.9830
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - acc: 0.5287 - loss: 2.9459 - val_acc: 0.5427 - val_loss: 2.4302
Epoch 8/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - acc: 0.5550 - loss: 2.3118 - val_acc: 0.5695 - val_loss: 2.0200
Epoch 9/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 15

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[[ 33  10   0   0   0   0   0]
 [ 32  56   0   0   1   1   0]
 [  1   3  81   0   0   0   0]
 [  6   0   0 102   0  38   0]
 [ 23   8   4  18  91  49  10]
 [  3   0   0  52   0  82   5]
 [  8   0   3  21   3  15  61]]
              precision    recall  f1-score   support

       Hello       0.31      0.77      0.44        43
        Meet       0.73      0.62      0.67        90
     NiceTMY       0.92      0.95      0.94        85
           I       0.53      0.70      0.60       146
        Name       0.96      0.45      0.61       203
         You       0.44      0.58      0.50       142
       Today       0.80      0.55      0.65       111

    accuracy                           0.62       820
   macro avg       0.67      0.66      0.63       820
weighted avg       0.71      0.62      0.63       820
